<a href="https://colab.research.google.com/github/nourhanhamdyebrahim123/LULC_change_Alexandria/blob/main/LULC_change_Alex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imagecodecs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 61.8 MB/s eta 0:00:00


In [ ]:
#!pip install tensorflow==2.13.0
import tensorflow as tf
print(tf.__version__)


2.19.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install segmentation_models generic_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.5 MB/s eta 0:00:00


In [ ]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping

from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, ConvLSTM2D, Input
from keras.models import Sequential, Model, load_model
import os
import tifffile as tif
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
#import segmentation_models as sm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
from sklearn.metrics import jaccard_score, f1_score, confusion_matrix, cohen_kappa_score
from datetime import datetime

In [ ]:
#data_dir = '/content/drive/My Drive/LULC_change_modeling/Dataset/'
data_dir = '/content/drive/My Drive/LULC_change_modeling_Alex/Dataset/'

In [ ]:


# Define the parameters
total_frames = 288  # Total number of frames (adjust according to your dataset)
frames_per_sequence = 4  # Number of frames per sequence
num_sequences = total_frames // frames_per_sequence  # Number of sequences
img_height, img_width = 256, 256  # Image dimensions
array_shape = (num_sequences, frames_per_sequence, img_height, img_width, 1)

# Initialize the array
x_test = np.zeros(array_shape, dtype=np.float32)



# Iterate over the frames and group them into sequences
# for seq_idx in range(num_sequences): # equal 15
#     sequence_dir = os.path.join(data_dir, f'sequence_{seq_idx+1:02d}')
paths = glob.glob(os.path.join(data_dir, '**', 'X_test', 'images'), recursive=True)


seq_idx = 0
# for path, subdirs, files in os.walk(data_dir):
#     #print(path)
#     dirname = path.split(os.path.sep)[-1]
#     #print("directory name,", dirname)
#     tilename = path.split(os.path.sep)[0]
#     #print("dir_tile", dirname)

#     if path == os.path.endswith('X_train\images'):
#        print ("in sub directory")

#     #if dirname == 'images':
#     if subdirectory_path:
#         print ("in sub directory")
#         print(subdirectory_path)

for path in paths:
        masks = os.listdir(path)
        print(masks)



        for frame_idx, mask_name in sorted(enumerate(masks), key=lambda x: x[1]):

            if mask_name.endswith(".tif"):   #Only read jpg images...
                mask = tif.imread(path+ "/" +mask_name)  #Read each image as RGB tif file
                if np.isnan(mask).any():
                    print("False")
                    print(path+"/"+mask_name)
                else:
                    print("True")
                #print ("whether Nan: ", np.isnan(image))
                mask[mask < 0] = 0.000001
                mask[mask == 0.0] = 0.000001
                mask[np.isnan(mask)] = 0.000001

                print("mask name is: ", mask_name)
                # plt.figure(figsize=(12,12))
                # plt.subplot(221)
                # plt.title(f'image {frame_idx+1}')
                # plt.axis('off')
                # plt.imshow(mask, cmap='gray')
                mask = np.expand_dims(mask, axis=-1) # for gray scale
                image_array = np.array(mask)
                # Insert the image into the array at the correct position
                print("seq_idex", seq_idx)
                print("fame_idex", frame_idx)
                x_test[seq_idx, frame_idx] = image_array

        print("sequence index", seq_idx)
        seq_idx += 1

# Print the final shape to confirm
print("Final array shape_x_test:", x_test.shape)

['image_1995_00.04.tif', 'image_2005_00.04.tif', 'image_2015_00.04.tif', 'image_2025_00.04.tif']
True
mask name is:  image_1995_00.04.tif
seq_idex 0
fame_idex 0
True
mask name is:  image_2005_00.04.tif
seq_idex 0
fame_idex 1
True
mask name is:  image_2015_00.04.tif
seq_idex 0
fame_idex 2
True
mask name is:  image_2025_00.04.tif
seq_idex 0
fame_idex 3
sequence index 0
['image_2025_00.05.tif', 'image_1995_00.05.tif', 'image_2015_00.05.tif', 'image_2005_00.05.tif']
True
mask name is:  image_1995_00.05.tif
seq_idex 1
fame_idex 1
True
mask name is:  image_2005_00.05.tif
seq_idex 1
fame_idex 3
True
mask name is:  image_2015_00.05.tif
seq_idex 1
fame_idex 2
True
mask name is:  image_2025_00.05.tif
seq_idex 1
fame_idex 0
sequence index 1
['image_2005_00.06.tif', 'image_2015_00.06.tif', 'image_1995_00.06.tif', 'image_2025_00.06.tif']
True
mask name is:  image_1995_00.06.tif
seq_idex 2
fame_idex 2
True
mask name is:  image_2005_00.06.tif
seq_idex 2
fame_idex 0
True
mask name is:  image_2015_00.0

In [ ]:
import numpy as np
import os
import glob
import tifffile as tif
import imagecodecs # Explicitly import imagecodecs

# Define parameters
total_frames = 288  # Adjust as needed
frames_per_sequence = 4
num_sequences = total_frames // frames_per_sequence
img_height, img_width = 256, 256
array_shape = (num_sequences, frames_per_sequence, img_height, img_width, 1)

x_test = np.zeros(array_shape, dtype=np.float32)

# Find all X_test image folders
paths = glob.glob(os.path.join(data_dir, '**', 'X_test', 'images'), recursive=True)

seq_idx = 0

for path in sorted(paths):
    masks = [m for m in os.listdir(path) if m.endswith('.tif')]

    # Sort by year extracted from filename (e.g., "image_1995_06.05.tif")
    masks.sort(key=lambda name: int(name.split('_')[1]))

    print(f"\n📁 Sequence folder: {path}")
    print(f"🧩 Sorted frames: {masks}")

    for frame_idx, mask_name in enumerate(masks):
        mask_path = os.path.join(path, mask_name)
        mask = tif.imread(mask_path)

        # Handle NaNs and negative values
        mask = np.nan_to_num(mask, nan=1e-6)
        mask[mask <= 0] = 1e-6

        mask = np.expand_dims(mask, axis=-1)
        x_test[seq_idx, frame_idx] = mask

        print(f"  ✅ {mask_name} → seq {seq_idx}, frame {frame_idx}")

    seq_idx += 1
    if seq_idx >= num_sequences:
        break

print("\n✅ Final array shape_x_test:", x_test.shape)


📁 Sequence folder: /content/drive/My Drive/LULC_change_modeling_Alex/Dataset/Sequence_01/X_test/images
🧩 Sorted frames: ['image_1995_00.00.tif', 'image_2005_00.00.tif', 'image_2015_00.00.tif', 'image_2025_00.00.tif']
  ✅ image_1995_00.00.tif → seq 0, frame 0
  ✅ image_2005_00.00.tif → seq 0, frame 1
  ✅ image_2015_00.00.tif → seq 0, frame 2
  ✅ image_2025_00.00.tif → seq 0, frame 3

📁 Sequence folder: /content/drive/My Drive/LULC_change_modeling_Alex/Dataset/Sequence_02/X_test/images
🧩 Sorted frames: ['image_1995_00.01.tif', 'image_2005_00.01.tif', 'image_2015_00.01.tif', 'image_2025_00.01.tif']
  ✅ image_1995_00.01.tif → seq 1, frame 0
  ✅ image_2005_00.01.tif → seq 1, frame 1
  ✅ image_2015_00.01.tif → seq 1, frame 2
  ✅ image_2025_00.01.tif → seq 1, frame 3

📁 Sequence folder: /content/drive/My Drive/LULC_change_modeling_Alex/Dataset/Sequence_03/X_test/images
🧩 Sorted frames: ['image_1995_00.02.tif', 'image_2005_00.02.tif', 'image_2015_00.02.tif', 'image_2025_00.02.tif']
  ✅ image_

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, ConvLSTM2D, BatchNormalization, Dense
# import segmentation_models as sm # Removed this import as it's causing the error

# Define your custom loss and metrics again (same as in training)
# If you don't have them right now, use dummy ones temporarily.
def total_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))  # Example: MSE

# Re-defining metrics without relying on segmentation_models
# Assuming these were the metrics used during training
def iou_score(y_true, y_pred, threshold=0.5):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / (union + tf.keras.backend.epsilon())

def f1_score(y_true, y_pred, threshold=0.5):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

metrics = [iou_score, f1_score]


def build_convlstm_model():
    image_input = Input(shape=(4, 256, 256, 1))

    x = ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same', return_sequences=True)(image_input)
    x = BatchNormalization()(x)
    x = ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='same', return_sequences=False)(x)
    x = BatchNormalization()(x)

    # The output layer should match the number of classes in your segmentation task.
    # Based on the later use of np.argmax with axis=3, it seems you have 4 classes.
    x = Dense(4, activation='softmax')(x)

    model = Model(inputs=image_input, outputs=x)
    # Compile the model with the same optimizer and loss function used during training
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=metrics)
    return model

# ---- Build and load weights ----
model = build_convlstm_model()

# Load weights (not the full model, to avoid version mismatch issues)
model.load_weights("/content/drive/My Drive/LULC_change_modeling_Alex/convlstm_time_series.hdf5")

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 256, 256, 1) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d (ConvLSTM2D)        │ (None, 4, 256, 256,    │       150,016 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 256, 256,    │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 256, 256, 32)   │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256, 256, 4)    │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261,252 (1020.52 KB)

 Trainable params: 261,060 (1019.77 KB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
#predicted_start = datetime.now()

predicted_image = model.predict(x_test)
print("shape of predicted_image before argmax is", predicted_image.shape) # Added print statement
predicted_image = np.argmax(predicted_image, axis=3).astype(np.uint32) # important to get categorized image
print("shape of predicted_image after argmax is", predicted_image.shape)
# predicted_image = np.expand_dims(predicted_image, axis=-1)  # shape now is (256, 256, 1) # This line seems incorrect based on expected shape after argmax


#predicted_stop = datetime.now()
#Execution time of the model
#Inference_time = predicted_stop-predicted_start
#print("Execution time of prediction is: ", Inference_time)

for k in range (len(predicted_image)):
    print("k value is:", k)
    print("shape of predicted_image[k] is", predicted_image[k].shape)
    #predicted_image = model.predict(x_test[k])
    #predicted_image = np.argmax(predicted_image[k], axis=3).astype(np.uint32) # important to get categorized image
    #predicted_image = np.argmax(predicted_image[k], axis=2).astype(np.uint32) # important to get categorized image



   # predicted_image = np.squeeze(predicted_image[k], axis=0)  # shape now is (256, 256, 1)
    # plt.figure(figsize=(12,12))
    # plt.subplot(221)
    # plt.title('Sequence0 Image')
    # plt.axis('off')
    # # plt.imshow(predicted_image, cmap='tab20')
    # plt.imshow(predicted_image, cmap='gray')
    #tif.imwrite('E:\Ph.D\Practical\Practical_LULC_change_prediction\My_trials for Convlstm\Practical_11-09-2024\Results\\prediction_'+str(k)+'.tif', predicted_image[k])
    img_name = f"{k:02}"
    tif.imwrite('/content/drive/My Drive/LULC_change_modeling_Alex/Results/prediction_' +img_name +'.tif', predicted_image[k])

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 751ms/step
shape of predicted_image before argmax is (72, 256, 256, 4)
shape of predicted_image after argmax is (72, 256, 256)
k value is: 0
shape of predicted_image[k] is (256, 256)
k value is: 1
shape of predicted_image[k] is (256, 256)
k value is: 2
shape of predicted_image[k] is (256, 256)
k value is: 3
shape of predicted_image[k] is (256, 256)
k value is: 4
shape of predicted_image[k] is (256, 256)
k value is: 5
shape of predicted_image[k] is (256, 256)
k value is: 6
shape of predicted_image[k] is (256, 256)
k value is: 7
shape of predicted_image[k] is (256, 256)
k value is: 8
shape of predicted_image[k] is (256, 256)
k value is: 9
shape of predicted_image[k] is (256, 256)
k value is: 10
shape of predicted_image[k] is (256, 256)
k value is: 11
shape of predicted_image[k] is (256, 256)
k value is: 12
shape of predicted_image[k] is (256, 256)
k value is: 13
shape of predicted_image[k] is (256, 256)
k value is: 14
shape of predicted_image[k] is (256, 256)


In [ ]:
########################## blend the predicted images #####################################################

from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/LULC_change_modeling_Alex/Results/'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 125.2 MB/s eta 0:00:00


In [ ]:
###### old ############

import numpy as np

def blend_images_sequentially(images, num_rows=8, num_cols=9):
    """
    Blend patches sequentially into a full image.

    Parameters:
        images (list or np.ndarray): List or array of shape (num_images, h, w, c) or (num_images, h, w)
        num_rows (int): Number of patch rows in the final image
        num_cols (int): Number of patch columns in the final image

    Returns:
        np.ndarray: Reconstructed blended image
    """
    num_images = len(images)
    assert num_images == num_rows * num_cols, f"Expected {num_rows*num_cols} images, got {num_images}"

    # Ensure consistent dimensions
    if images[0].ndim == 3:
        h, w, c = images[0].shape
    elif images[0].ndim == 2:
        h, w = images[0].shape
        c = 1
        images = [np.expand_dims(img, axis=-1) for img in images]
    else:
        raise ValueError("Unsupported image dimensions")

    # Build each row
    blended_rows = []
    for r in range(num_rows):
        start_idx = r * num_cols
        end_idx = start_idx + num_cols
        row_images = images[start_idx:end_idx]
        blended_row = np.hstack(row_images)
        blended_rows.append(blended_row)

    # Stack rows vertically
    final_image = np.vstack(blended_rows)

    # If grayscale, remove redundant channel dimension
    if c == 1:
        final_image = np.squeeze(final_image, axis=-1)

    return final_image


In [ ]:
import os
import numpy as np
import tifffile as tif
# Get all .tif files that start with 'prediction_'
predicted_files = sorted([f for f in os.listdir(data_dir) if f.startswith('prediction_') and f.endswith('.tif')])

# Load them into a numpy array
predicted_image = np.array([tif.imread(os.path.join(data_dir, f)) for f in predicted_files])

print("Loaded predicted_image shape:", predicted_image.shape)


Loaded predicted_image shape: (72, 256, 256)


In [ ]:
blended_predicted = blend_images_sequentially(predicted_image, num_rows=8, num_cols=9)
tif.imwrite('/content/drive/My Drive/LULC_change_modeling_Alex/Results/predicted_blended_2.tif', blended_predicted)

In [ ]:
predicted_start = datetime.now()

predicted_image = model.predict(x_test)
print("shape of predicted_image before argmax is", predicted_image.shape) # Added print statement
predicted_image = np.argmax(predicted_image, axis=3).astype(np.uint32) # important to get categorized image
print("shape of predicted_image after argmax is", predicted_image.shape)
# predicted_image = np.expand_dims(predicted_image, axis=-1)  # shape now is (256, 256, 1) # This line seems incorrect based on expected shape after argmax


predicted_stop = datetime.now()
#Execution time of the model
Inference_time = predicted_stop-predicted_start
print("Execution time of prediction is: ", Inference_time)

for k in range (len(predicted_image)):
    print("k value is:", k)
    print("shape of predicted_image[k] is", predicted_image[k].shape)
    #predicted_image = model.predict(x_test[k])
    #predicted_image = np.argmax(predicted_image[k], axis=3).astype(np.uint32) # important to get categorized image
    #predicted_image = np.argmax(predicted_image[k], axis=2).astype(np.uint32) # important to get categorized image



   # predicted_image = np.squeeze(predicted_image[k], axis=0)  # shape now is (256, 256, 1)
    # plt.figure(figsize=(12,12))
    # plt.subplot(221)
    # plt.title('Sequence0 Image')
    # plt.axis('off')
    # # plt.imshow(predicted_image, cmap='tab20')
    # plt.imshow(predicted_image, cmap='gray')
    #tif.imwrite('E:\Ph.D\Practical\Practical_LULC_change_prediction\My_trials for Convlstm\Practical_11-09-2024\Results\\prediction_'+str(k)+'.tif', predicted_image[k])
    img_name = f"{k:02}"
    tif.imwrite('/content/drive/My Drive/LULC_change_modeling_Alex/Results/prediction_' +img_name +'.tif', predicted_image[k])


    # #y_test[k] = np.squeeze(y_test[k], axis=0)  # shape now is (256, 256, 1)
    # plt.figure(figsize=(12,12))
    # plt.subplot(221)
    # plt.title('Sequence0 Image')
    # plt.axis('off')
    # plt.imshow(y_test[k], cmap='gray')
    #tif.imwrite('/content/drive/My Drive/LULC_change_modeling_Alex/y_test_'+ img_name +'.tif', y_test[k])
    #tif.imwrite('E:\Ph.D\Practical\Practical_LULC_change_prediction\My_trials for Convlstm\Practical_11-09-2024\Results\\y_test_'+str(k)+'.tif', y_test[k])



def blend_images_sequentially(images):
    # Assuming images is a list or numpy array of shape (num_images, height, width, channels) or (num_images, height, width)
    assert len(images) == 72  # Ensure you have 72 images

    # Check the number of dimensions in the image data
    if images[0].ndim == 3:
        height, width, channels = images[0].shape
    elif images[0].ndim == 2:
        height, width = images[0].shape
        channels = 1 # Assume grayscale if no channel dimension
    else:
        raise ValueError("Unsupported image dimensions")

    # Create an empty list to hold the blended rows
    blended_rows = []

    # Process images in groups of 5
    for i in range(0, 15, 5):
        # Take 5 images for this row
        row_images = images[i:i+5]

        # Blend them smoothly across the horizontal direction
        # Ensure consistent number of dimensions for stacking
        if images[0].ndim == 2:
             row_images = [np.expand_dims(img, axis=-1) for img in row_images] # Add channel dimension for hstack
        blended_row = np.hstack(row_images)  # Concatenate images horizontally
        blended_rows.append(blended_row)

    # Now, stack the three rows vertically
    final_image = np.vstack(blended_rows)

    return final_image

blended_actual = blend_images_sequentially(y_test)
tif.imwrite('/content/drive/My Drive/LULC_change_modeling/Results/y_test_blended.tif', blended_actual)


blended_predicted = blend_images_sequentially(predicted_image)
tif.imwrite('/content/drive/My Drive/LULC_change_modeling/Results/predicted_blended.tif', blended_predicted)

blended_actual_flatten = blended_actual.flatten()
blended_predicted_flatten = blended_predicted.flatten()

f1_score_value = f1_score(blended_actual_flatten, blended_predicted_flatten, average="weighted")
print("f1 score value of is: ", f1_score_value)

jaccard_index = jaccard_score(blended_actual_flatten, blended_predicted_flatten, average="weighted")
print("jaccard index value of is: ", jaccard_index)



conf_matrix = confusion_matrix(blended_actual_flatten, blended_predicted_flatten)
print("Confusion Matrix:", conf_matrix)
print("Confusion Matrix with Labels:")
class_labels = np.unique(blended_actual_flatten).astype(np.uint8)  # Get unique class labels from ground truth


for i, label in enumerate(class_labels):
    print(f"Class {label}: {conf_matrix[i]}")




kappa_value = cohen_kappa_score(blended_actual_flatten, blended_predicted_flatten)
print(f"Cohen's Kappa Value: {kappa_value:.4f}")



def calculate_class_kappa_scores_image(y_true, y_pred, num_classes):
    kappa_scores = []

    # Flatten the images into 1D arrays
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    # Iterate over each class and calculate one-vs-rest kappa
    for class_label in range(num_classes):
        # Create binary labels for one-vs-rest
        y_true_binary = np.where(y_true_flat == class_label, 1, 0)
        y_pred_binary = np.where(y_pred_flat == class_label, 1, 0)

        # Calculate kappa score for this class
        kappa = cohen_kappa_score(y_true_binary, y_pred_binary)
        kappa_scores.append(kappa)

        print(f"Class {class_label}: Kappa = {kappa:.3f}")

    return kappa_scores

kappa_scores = calculate_class_kappa_scores_image(blended_actual_flatten, blended_predicted_flatten, 4)

3/3 ━━━━━━━━━━━━━━━━━━━━ 336s 86s/step
shape of predicted_image before argmax is (72, 256, 256, 4)
shape of predicted_image after argmax is (72, 256, 256)
Execution time of prediction is:  0:05:36.384688
k value is: 0
shape of predicted_image[k] is (256, 256)


NameError: name 'y_test' is not defined

In [ ]:
###########################################################################################################################################################################################################